# MNIST 手寫數字辨識 (MNIST_Autoencoder_TF)

2017/07/21   
徐仕杰

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3



## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. Modeling](#開始Deep Learning)  
-  [3. Training](#reference)
-  [4. Tensorboard](#)

<a id='import'></a>
## Import Package & Functions

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import matplotlib.pyplot as plt
from PIL import Image
import warnings
from sklearn import preprocessing
from functools import reduce

In [2]:
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

In [5]:
def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)
def bias_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)

In [6]:
def kl_div(rho, rho_hat):
    invrho = tf.subtract(tf.constant(1.), rho)
    invrhohat = tf.subtract(tf.constant(1.), rho_hat)
    logrho = tf.add(logfunc(rho,rho_hat), logfunc(invrho, invrhohat))
    return logrho
    
def logfunc(x, x2):
    return tf.multiply( x, tf.log(tf.div(x,x2)))

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [7]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [8]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [9]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='Modeling'></a>
## 2. Modeling


** What is autoencoder ? **

** autoencoder可以幹嘛？ **


- Autoencoder 
    -  [Parameters](#定義參數) 
    -  [Autoencoder modeling - Network Structure](#設計一個) 
    -  [Autoencoder modeling - Loss Function & Optimizer](#選一個loss) 
    -  [Autoencoder training](#選一個o) 
    -  [Autoencoder - test](#開始執行)   
- Classifier       
    -  [Classifier modeling - Network Structure](#算一下正)  
    -  [Classifier modeling - loss function and Optimizer](#)
    -  [Classifier training](#)
    -  [Classifier testing](#)

** Parameters **

In [10]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, 784], name="X")
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")

** Autoencoder modeling - Network Structure **

encoded_neural = 30
def build_sae():
    W_e_1 = weight_variable([784, 1000], "w_e_1")
    b_e_1 = bias_variable([1000], "b_e_1")
    h_e_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W_e_1), b_e_1))

    W_e_2 = weight_variable([1000, 500], "w_e_2")
    b_e_2 = bias_variable([500], "b_e_2")
    h_e_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_1, W_e_2), b_e_2))
    
    W_e_3 = weight_variable([500, 250], "w_e_3")
    b_e_3 = bias_variable([250], "b_e_3")
    h_e_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_2, W_e_3), b_e_3))
    
    W_e_4 = weight_variable([250, encoded_neural], "w_e_4")
    b_e_4 = bias_variable([encoded_neural], "b_e_4")
    h_e_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_3, W_e_4), b_e_4))

    W_d_1 = weight_variable([encoded_neural, 250], "w_d_1")
    b_d_1 = bias_variable([250], "b_d_1")
    h_d_1 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_4, W_d_1), b_d_1))
    
    W_d_2 = weight_variable([250, 500], 'w_d_2')
    b_d_2 = bias_variable([500], "b_d_2")
    h_d_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_1, W_d_2), b_d_2))

    W_d_3 = weight_variable([500, 1000], "w_d_3")
    b_d_3 = bias_variable([1000], "b_d_3")
    h_d_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_2, W_d_3), b_d_3))
    
    W_d_4 = weight_variable([1000, 784], "w_d_4")
    b_d_4 = bias_variable([784], "b_d_4")
    h_d_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_3, W_d_4), b_d_4))
    h = [h_e_1, h_e_2, h_e_3, h_e_4, h_d_1, h_d_2, h_d_3, h_d_4]
    w = [W_e_1, W_e_2,W_e_3, W_e_4, W_d_1, W_d_2, W_d_3,  W_d_4]
    
    return h, w, h_e_4, h_d_4

h, w, encoded, x_reconstruct = build_sae()

In [11]:
encoded_neural = 30
def build_sae():
    W_e_1 = weight_variable([784, 500], "w_e_1")
    b_e_1 = bias_variable([500], "b_e_1")
    h_e_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W_e_1), b_e_1))

    W_e_2 = weight_variable([500, 250], "w_e_2")
    b_e_2 = bias_variable([250], "b_e_2")
    h_e_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_1, W_e_2), b_e_2))

    W_e_4 = weight_variable([250, encoded_neural], "w_e_4")
    b_e_4 = bias_variable([encoded_neural], "b_e_4")
    h_e_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_2, W_e_4), b_e_4))

    W_d_1 = weight_variable([encoded_neural, 250], "w_d_1")
    b_d_1 = bias_variable([250], "b_d_1")
    h_d_1 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_4, W_d_1), b_d_1))

    W_d_3 = weight_variable([250, 500], "w_d_3")
    b_d_3 = bias_variable([500], "b_d_3")
    h_d_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_1, W_d_3), b_d_3))
    
    W_d_4 = weight_variable([500, 784], "w_d_4")
    b_d_4 = bias_variable([784], "b_d_4")
    h_d_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_3, W_d_4), b_d_4))
    h = [h_e_1, h_e_2, h_e_4, h_d_1, h_d_3, h_d_4]
    w = [W_e_1, W_e_2, W_e_4, W_d_1, W_d_3,  W_d_4]
    
    return h, w, h_e_4, h_d_4

h, w, encoded, x_reconstruct = build_sae()

** Autoencoder modeling - Loss Function & Optimizer **

In [12]:
alpha = 5e-6
beta = 7.5e-5
kl_div_loss = reduce(lambda x, y: x + y, map(lambda x: tf.reduce_sum(kl_div(0.02, tf.reduce_mean(x,0))), h))
l2_loss = reduce(lambda x, y: x + y, map(lambda x: tf.nn.l2_loss(x), w))
loss_auto = tf.reduce_mean(tf.pow(x_reconstruct - X, 2)) #+ alpha * l2_loss #+ beta * kl_div_loss

optimizer = tf.train.AdamOptimizer(0.01).minimize(loss_auto)


** Classifier modeling - Network Structure **

In [13]:
def build_classfier(encoded):
    W_cf_1 = weight_variable([30, 10], "W_cf_1")
    b_cf_1 = bias_variable([10], "b_cf_1")
    h_cf_1 = tf.add(tf.matmul(encoded, W_cf_1), b_cf_1)
    h_cf_1_sf = tf.nn.softmax(h_cf_1)
    return [h_cf_1_sf], [W_cf_1,b_cf_1], h_cf_1, h_cf_1_sf

_, trainable_vars, Y, Y_softmax = build_classfier(encoded)
#correct_prediction = tf.equal(Y_softmax_argmax, tf.argmax(Y_, 1))
correct_pred = tf.equal(tf.argmax(Y_softmax, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))

** Classifier modeling - loss function and Optimizer **

In [14]:
loss_cla = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Y))
#optimizer_cla = tf.train.RMSPropOptimizer(0.01).minimize(loss_cla,var_list=trainable_vars)
optimizer_cla = tf.train.RMSPropOptimizer(0.01).minimize(loss_cla)

## 3. Training

** Autoencoder training **

In [ ]:
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)


In [20]:
epoch = 15
batch_size = 500
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        _, loss_v= sess.run([optimizer, loss_auto], feed_dict={X: batch_x})
    updateProgress('epoch:{x0}, batch:{x4} loss:{x3}'.format(x0=ep,x3=loss_v,x4=i))
    print()


epoch:0, batch:100 loss:0.019495276734232903
epoch:1, batch:100 loss:0.018572742119431496
epoch:2, batch:100 loss:0.01892249286174774
epoch:3, batch:100 loss:0.01788116991519928
epoch:4, batch:100 loss:0.017328714951872826
epoch:5, batch:100 loss:0.017409678548574448
epoch:6, batch:100 loss:0.01654631271958351
epoch:7, batch:100 loss:0.01644429937005043
epoch:8, batch:100 loss:0.01664084754884243
epoch:9, batch:100 loss:0.014998780563473701
epoch:10, batch:100 loss:0.015366114675998688
epoch:11, batch:100 loss:0.015432391315698624
epoch:12, batch:100 loss:0.015046445652842522
epoch:13, batch:100 loss:0.014436179772019386
epoch:14, batch:100 loss:0.014812003821134567
epoch:15, batch:100 loss:0.014089689590036869


** Autoencoder - test mnist **

In [21]:
showX(test_X[:20])
batch_tx = test_X[:20]
decoded_imgs = sess.run([x_reconstruct], feed_dict={X: batch_tx})

decoded_imgs_1 = np.array(decoded_imgs)
decoded_imgs_1 = decoded_imgs_1.reshape(20,784)
print('')
showX(decoded_imgs_1[:20])

![](img/fail.png)

** Autoencoder - test other picture **

In [22]:
warnings.filterwarnings('ignore')
piclist = ['img/ae1.jpg','img/ae2.jpg','img/ae3.jpg','img/ae4.jpeg','img/ae5.jpeg']
batch_tx=[]
min_max_scaler = preprocessing.MinMaxScaler()
normalize_scaler = preprocessing.StandardScaler()
for pic in piclist:
    im = Image.open(pic).convert('L') #Can be many different formats.
    #display(im)
    im = im.resize((28,28))
    display(im)
    #pix = im.load()
    pixels = list(im.getdata())
    pixels = np.array(pixels).reshape(784)
    pixels_scaled = normalize_scaler.fit_transform(pixels.reshape((-1, 1)))
    batch_tx.append(pixels_scaled.reshape((784)))
    
decoded_img_test = sess.run([x_reconstruct], feed_dict={X: batch_tx})
decoded_img_test = np.array(decoded_img_test)
decoded_img_test = decoded_img_test.reshape(-1,784)
showX(decoded_img_test[:20])

** Classifier training **

In [23]:
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, loss_v, acc_v= sess.run([optimizer_cla, loss_cla, accuracy], feed_dict={X: batch_x, Y_:batch_y})
    loss_v2, acc_v2= sess.run([loss_cla, accuracy], feed_dict={X: validation_X , Y_: validation_Y})
    updateProgress('Ep:{x0}, Bch:{x4}, Tr: loss:{x5}, acc:{x6}, Vd: loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc_v2,x3=loss_v2,x4=i,x6=acc_v,x5=loss_v))
    print()



Ep:0, Bch:390, Tr: loss:0.23164501786231995, acc:0.9375, Vd: loss:0.18050619959831238, acc:0.9484999775886536
Ep:1, Bch:390, Tr: loss:0.1820179522037506, acc:0.96875, Vd: loss:0.15137505531311035, acc:0.960099995136261
Ep:2, Bch:390, Tr: loss:0.0717480331659317, acc:0.9765625, Vd: loss:0.1374199092388153, acc:0.9623000025749207
Ep:3, Bch:390, Tr: loss:0.015384380705654621, acc:0.9921875, Vd: loss:0.09729854762554169, acc:0.9760000109672546
Ep:4, Bch:390, Tr: loss:0.109468474984169, acc:0.9609375, Vd: loss:0.10849763453006744, acc:0.9729999899864197
Ep:5, Bch:390, Tr: loss:0.03278045728802681, acc:0.984375, Vd: loss:0.09915957599878311, acc:0.9768000245094299
Ep:6, Bch:390, Tr: loss:0.09993363916873932, acc:0.984375, Vd: loss:0.09416057914495468, acc:0.9775999784469604
Ep:7, Bch:390, Tr: loss:0.02570577897131443, acc:0.9921875, Vd: loss:0.11107805371284485, acc:0.9758999943733215
Ep:8, Bch:390, Tr: loss:0.033155348151922226, acc:0.9921875, Vd: loss:0.1046699583530426, acc:0.976599991321

** Classifier testing **

In [25]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = Y_softmax.eval(feed_dict={X: train_X[21:30]})
pred_tmp_df = pd.DataFrame(pred_tmp)
_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])
_ = correct_pred.eval({X: train_X[21:30] , Y_: train_Y[21:30]})
print('Training 第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [0 9 1 1 2 4 3 2 7]


answer:  [0 9 1 1 2 4 3 2 7]
Training 第21~30筆資料的比對結果:       0
0  True
1  True
2  True
3  True
4  True
5  True
6  True
7  True
8  True


In [28]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = Y_softmax.eval(feed_dict={X: test_X[:100]})
pred_tmp_df = pd.DataFrame(pred_tmp)
_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(test_X[:100])
batch_tx = test_X[:100]
decoded_imgs = sess.run([x_reconstruct], feed_dict={X: batch_tx})
decoded_imgs_1 = np.array(decoded_imgs)
decoded_imgs_1 = decoded_imgs_1.reshape(-1,784)
showX(decoded_imgs_1)
print('answer: ',train_y[:100])
_ = correct_pred.eval({X: test_X[:100] , Y_: test_Y[:100]})
print('Testing 第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3 7 9 6 4 3 0 7 0 2 9
 1 7 3 2 9 7 7 6 2 7 8 4 7 3 6 1 3 6 9 3 1 4 1 7 6 9]


answer:  [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1]
Testing 第21~30筆資料的比對結果:        0
0   True
1   True
2   True
3   True
4   True
5   True
6   True
7   True
8   True
9   True
10  True
11  True
12  True
13  True
14  True
15  True
16  True
17  True
18  True
19  True
20  True
21  True
22  True
23  True
24  True
25  True
26  True
27  True
28  True
29  True
..   ...
70  True
71  True
72  True
73  True
74  True
75  True
76  True
77  True
78  True
79  True
80  True
81  True
82  True
83  True
84  True
85  True
86  True
87  True
88  True
89  True
90  True
91  True
92  True
93  True
94  True
95  True
96  True
97  True
98  True
99  True

[100 rows x 1 columns]


In [30]:
acc = sess.run(accuracy, feed_dict={X: test_X , Y_: test_Y})
print('Accuracy:',acc)

Accuracy: 0.9775


### Tensor Borad

In [ ]:
LOG_DIR = 'tb-log0'
input_data = train_X[:5000]
input_y = train_y[:5000]
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
#把圖片的label存到metadata.tsv
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(input_y)):
        c=input_y[row]
        metadata_file.write('{}\n'.format(c))

In [ ]:
#Save variable
Input_Var = tf.Variable(input_data, name='Input_var')
tmp_aevalue = encoded.eval({X: input_data})
encoded_Var = tf.Variable(encoded.eval({X: input_data}), name='encoded_var')
x_reconstruct_Var = tf.Variable(x_reconstruct.eval({X: input_data}), name='x_reconstruct_Var')

saver = tf.train.Saver([Input_Var,encoded_Var,x_reconstruct_Var])
sess.run(Input_Var.initializer)
sess.run(encoded_Var.initializer)
sess.run(x_reconstruct_Var.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'autoencoder.ckpt'))

#在config裡面用一個embedding關聯 tensor & its metadata
config = projector.ProjectorConfig()
# One can add multiple embeddings.

embedding1 = config.embeddings.add()
embedding1.tensor_name = Input_Var.name
embedding1.metadata_path = metadata_file_path

embedding2 = config.embeddings.add()
embedding2.tensor_name = encoded_Var.name
embedding2.metadata_path = metadata_file_path

embedding3 = config.embeddings.add()
embedding3.tensor_name = x_reconstruct_Var.name
embedding3.metadata_path = metadata_file_path

#embedding4 = config.embeddings.add()
#embedding4.tensor_name = Out_Var.name
#embedding4.metadata_path = metadata_file_path

#embedding.sprite.image_path = os.path.join(LOG_DIR, 'img/mnist_10k_sprite.png')
# Specify the width and height of a single thumbnail.
#embedding.sprite.single_image_dim.extend([28, 28])



# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


In [ ]:
#!tensorboard --logdir=tb-log0 --port=6006

** epoch 15 **
![epoch15](img/ae_emd_op15.png)

** epoch 30 **
![epoch15](img/ae_emd_op30.png)

** epoch 60 **
![epoch15](img/ae_emd_op60.png)

** epoch 150 **
![epoch15](img/ae_emd_op150.png)